In [ ]:
import pickle
import os
import numpy as np

## Data splitting

In [ ]:
with open('30frameDatasetLargeRect.pkl','rb') as f:
    dataset = pickle.load(f)

In [ ]:
x = dataset['features']
np.array(x).shape

In [ ]:
y = dataset['labels']
y = []
for label in dataset['labels']:
    if(label == 'correct'):
        y.append(1)
    if(label == 'incorrect'):
        y.append(0)

In [ ]:
# dataset['labels'] = y

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(np.array(dataset['features']),np.array(dataset['labels']), test_size=0.25, random_state=200)

## Training

In [ ]:
from keras.layers import Dense, ConvLSTM2D, MaxPooling3D, TimeDistributed, Dropout, Flatten
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [ ]:
def create_model():
    model = Sequential()

    model.add(ConvLSTM2D(filters=4, kernel_size=(3,3), activation='tanh', data_format='channels_last', recurrent_dropout=0.2, return_sequences=True,
                         input_shape = (30,64,64,3)))
    model.add(MaxPooling3D(pool_size=(1,2,2),padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters=8, kernel_size=(3,3), activation='tanh', data_format='channels_last', recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1,2,2),padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters=14, kernel_size=(3,3), activation='tanh', data_format='channels_last', recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1,2,2),padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters=14, kernel_size=(3,3), activation='tanh', data_format='channels_last', recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1,2,2),padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters=16, kernel_size=(3,3), activation='tanh', data_format='channels_last', recurrent_dropout=0.2, return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1,2,2),padding='same', data_format='channels_last'))
    
    model.add(Flatten())

    model.add(Dense(2, activation='softmax'))

    model.summary()
    return model

In [ ]:
model = create_model()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint


# Define Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)

# Define ModelCheckpoint Callback to save the model at the end of training
model_checkpoint_callback = ModelCheckpoint(filepath='30frame_best_model_large_box_Anish.h5', save_best_only=True, save_weights_only=False, monitor='val_loss', mode='min')

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# Train the model with validation data and callbacks
model.fit(xtrain, ytrain, epochs=50, batch_size=4, shuffle=True, callbacks=[early_stopping_callback, model_checkpoint_callback], validation_split=0.2)




In [29]:
from keras.models import load_model

# Load the best model saved during training
best_model = load_model('30frame_best_model_large_box_Anish.h5')

In [30]:
ypred = best_model.predict(xtest)

2/2 [==============================] - 1s 194ms/step


In [31]:
# Define the threshold
threshold = 0.5

# Convert sigmoid outputs to binary values
ypred = np.argmax(ypred,axis=1)
ypred

array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [32]:
from sklearn.metrics import *

confusion_matrix(ytest,ypred)

array([[25,  5],
       [13,  9]], dtype=int64)

In [33]:
accuracy_score(ytest,ypred)

0.6538461538461539